## A python stream generating application

In [23]:
from pyspark import SparkContext

-------------------------------------------
Time: 2018-05-07 15:51:14
-------------------------------------------

-------------------------------------------
Time: 2018-05-07 15:51:16
-------------------------------------------

-------------------------------------------
Time: 2018-05-07 15:51:18
-------------------------------------------

-------------------------------------------
Time: 2018-05-07 15:51:20
-------------------------------------------

-------------------------------------------
Time: 2018-05-07 15:51:22
-------------------------------------------

-------------------------------------------
Time: 2018-05-07 15:51:24
-------------------------------------------

-------------------------------------------
Time: 2018-05-07 15:51:26
-------------------------------------------

-------------------------------------------
Time: 2018-05-07 15:51:28
-------------------------------------------



In [22]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

spark = SparkSession \
    .builder \
    .appName("StructuredNetworkWordCount") \
    .getOrCreate()

# Create DataFrame representing the stream of input lines from connection to localhost:9999
lines = spark \
    .readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 9999) \
    .load()

# Split the lines into words
words = lines.select(
   explode(
       split(lines.value, " ")
   ).alias("word")
)

# Generate running word count
wordCounts = words.groupBy("word").count()

query = wordCounts \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .start()

query.awaitTermination()

-------------------------------------------
Time: 2018-05-07 15:48:44
-------------------------------------------

-------------------------------------------
Time: 2018-05-07 15:48:46
-------------------------------------------

-------------------------------------------
Time: 2018-05-07 15:48:48
-------------------------------------------

-------------------------------------------
Time: 2018-05-07 15:48:50
-------------------------------------------

-------------------------------------------
Time: 2018-05-07 15:48:52
-------------------------------------------

-------------------------------------------
Time: 2018-05-07 15:48:54
-------------------------------------------

-------------------------------------------
Time: 2018-05-07 15:48:56
-------------------------------------------

-------------------------------------------
Time: 2018-05-07 15:48:58
-------------------------------------------

-------------------------------------------
Time: 2018-05-07 15:49:00
----------

KeyboardInterrupt: 

-------------------------------------------
Time: 2018-05-07 15:49:30
-------------------------------------------

-------------------------------------------
Time: 2018-05-07 15:49:32
-------------------------------------------

-------------------------------------------
Time: 2018-05-07 15:49:34
-------------------------------------------

-------------------------------------------
Time: 2018-05-07 15:49:36
-------------------------------------------

-------------------------------------------
Time: 2018-05-07 15:49:38
-------------------------------------------

-------------------------------------------
Time: 2018-05-07 15:49:40
-------------------------------------------

-------------------------------------------
Time: 2018-05-07 15:49:42
-------------------------------------------

-------------------------------------------
Time: 2018-05-07 15:49:44
-------------------------------------------

-------------------------------------------
Time: 2018-05-07 15:49:46
----------